# Ensembl genes table extraction EDA

This notebook is useful for development as well as exploratory data analysis on the extracted tables.
It is currently automically executed and saved as part of exports using `papermill`.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from ensembl_genes import ensembl_genes

In [3]:
# parameters cell
species = "human"
release = "104"

In [4]:
# Parameters
species = "rat"
release = "104"


In [5]:
ensg = ensembl_genes.Ensembl_Gene_Queries(release=release, species=species)
ensg.connection_url

'mysql+mysqlconnector://anonymous@ensembldb.ensembl.org:3306/rattus_norvegicus_core_104_6'

In [6]:
database = ensg.database
database

'rattus_norvegicus_core_104_6'

## Extract data

## gene attrib counts

In [7]:
ensg.run_query("gene_attrib_counts").head(15)

,attrib_type_id,code,name,description,attrib_type_count,attrib_type_examples
0,142,GeneGC,Gene GC,Percentage GC content for this gene,32883,"40.15, 51.97, 44.36, 52.04, 32.61, 38.86, 45.3..."
1,4,name,Name,Alternative/long name,2610,"Rn60_20_0029.3, Bag6, Tspyl2, Rn60_X_0773.5, P..."
2,52,author,Author,Group resonsible for Vega annotation,2610,Havana
3,53,author_email,Author email address,Author email address,2610,vega@sanger.ac.uk
4,380,havana_cv,Havana CV term,Controlled vocabulary terms from Havana,604,"fragmented locus, overlapping locus, reference..."
5,123,hidden_remark,Hidden Remark,None,124,"confirm experimentally, ASB_protein_coding, fr..."
6,3,synonym,Synonym,None,121,"Gpr54, SPP-1;, sialoprotein, Vglut3, Hlals, RG..."
7,382,NoTransRefError,No translations due to reference error,This gene is believed to include protein codin...,27,1
8,54,remark,Remark,Annotation remark,4,"Selenoprotein, polymorphic pseudogene no stop"
9,1,embl_acc,European Nucleotide Archive (was EMBL) accession,ENA,0,None


## genes

In [8]:
ensg.gene_df.head()

,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source,gene_biotype,gene_description,ensembl_source,ensembl_created_date,ensembl_modified_date,...,chromosome,seq_region_exc_type,seq_region,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,ensembl_representative_gene_id
0,ENSRNOG00000000001,5,AABR07013255.1,Clone_based_ensembl_gene,AABR07013255.1,pseudogene,None,ensembl,2009-07-29 15:36:02,2015-04-02 16:53:59,...,2,None,2,230660664,230662084,1,True,NaN,None,ENSRNOG00000000001
1,ENSRNOG00000000007,7,Gad1,RGD,2652,protein_coding,glutamate decarboxylase 1 [Source:RGD Symbol;A...,ensembl,2009-07-29 15:36:02,2015-04-02 16:53:59,...,3,None,3,56861396,56902157,1,True,NaN,None,ENSRNOG00000000007
2,ENSRNOG00000000008,7,Alx4,RGD,1310201,protein_coding,ALX homeobox 4 [Source:RGD Symbol;Acc:1310201],ensembl,2009-07-29 15:36:02,2015-04-02 16:53:59,...,3,None,3,82548959,82585531,1,True,NaN,None,ENSRNOG00000000008
3,ENSRNOG00000000009,5,Tmco5b,RGD,1561237,protein_coding,transmembrane and coiled-coil domains 5B [Sour...,ensembl,2009-07-29 15:36:02,2015-04-02 16:53:59,...,3,None,3,104749051,104765436,1,True,NaN,None,ENSRNOG00000000009
4,ENSRNOG00000000010,5,Cbln1,RGD,1562813,protein_coding,cerebellin 1 precursor [Source:RGD Symbol;Acc:...,ensembl,2009-07-29 15:36:02,2009-07-29 15:36:02,...,19,None,19,20607507,20611316,1,True,NaN,None,ENSRNOG00000000010


In [9]:
# clone-based genes no longer get a symbol and are filled with the stable ID
# https://www.ensembl.info/2021/03/15/retirement-of-clone-based-gene-names/
ensg.gene_df.query("gene_symbol == ensembl_gene_id").head(2)

,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source,gene_biotype,gene_description,ensembl_source,ensembl_created_date,ensembl_modified_date,...,chromosome,seq_region_exc_type,seq_region,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,ensembl_representative_gene_id
415,ENSRNOG00000000885,5,ENSRNOG00000000885,None,None,protein_coding,autism susceptibility candidate 2-like [Source...,ensembl,2009-07-29 15:36:02,2009-07-29 15:36:02,...,12,None,12,28212333,28251816,1,True,NaN,None,ENSRNOG00000000885
420,ENSRNOG00000000891,5,ENSRNOG00000000891,None,None,protein_coding,zinc finger protein 958 [Source:MGI Symbol;Acc...,ensembl,2009-07-29 15:36:02,2012-11-09 06:35:19,...,12,None,12,4737817,4741997,1,True,NaN,None,ENSRNOG00000000891


In [10]:
# which external database the gene symbol derives from versus the ensembl source
pd.crosstab(
    ensg.gene_df.ensembl_source,
    ensg.gene_df.gene_symbol_source_db.fillna("missing (clone-based)"),
    margins=True,
)

gene_symbol_source_db,Clone_based_ensembl_gene,EntrezGene,MGI,RFAM,RGD,miRBase,missing (clone-based),All
ensembl_source,,,,,,,,
ensembl,8024,3,73,2561,19182,6,387,30236
ensembl_havana,102,0,0,0,1560,0,0,1662
havana,855,1,2,0,89,0,1,948
insdc,24,0,0,0,13,0,0,37
All,9005,4,75,2561,20844,6,388,32883


In [11]:
ensg.gene_df.coord_system.value_counts().head(10)

chromosome    32623
scaffold        260
Name: coord_system, dtype: int64

In [12]:
ensg.gene_df.gene_biotype.value_counts().head(10)

protein_coding            22250
lincRNA                    3090
snoRNA                     1654
miRNA                      1588
snRNA                      1475
pseudogene                  799
processed_pseudogene        728
misc_RNA                    524
rRNA                        329
unprocessed_pseudogene      126
Name: gene_biotype, dtype: int64

In [13]:
ensg.gene_df.seq_region_exc_type.value_counts(dropna=False)

NaN    32883
Name: seq_region_exc_type, dtype: int64

In [14]:
ensg.gene_df.mhc.value_counts()

Series([], Name: mhc, dtype: int64)

In [15]:
len(ensg.gene_df)

32883

## alternative gene alleles

Related:

- [OTP: Origin of genes_with_non_reference_ensembl_ids.tsv](https://github.com/opentargets/platform/issues/702)
- [biostars: map between different assemblies of one ensembl release](https://www.biostars.org/p/143956/)
- using `attrib_type.code = "non_ref"` for `primary_assembly` doesn't appear to return any results

In [16]:
ensg.alt_allele_df.head()

,ensembl_gene_id,alt_allele_group_id,alt_allele_is_representative,primary_assembly,seq_region,alt_allele_attrib,ensembl_created_date,ensembl_representative_gene_id,is_representative_gene,representative_gene_method
alt_allele_group_id,,,,,,,,,,


In [17]:
# looks like non_ref isn't set for human genes
query = '''
SELECT *
FROM gene_attrib
LEFT JOIN attrib_type
  ON gene_attrib.attrib_type_id = attrib_type.attrib_type_id
WHERE attrib_type.code = "non_ref"
LIMIT 5
'''
pd.read_sql(sql=query, con=ensg.connection_url)

,gene_id,attrib_type_id,value,attrib_type_id,code,name,description


In [18]:
ensg.alt_allele_df.alt_allele_attrib.value_counts()

Series([], Name: alt_allele_attrib, dtype: int64)

In [19]:
ensg.alt_allele_df.query("is_representative_gene").representative_gene_method.value_counts()

Series([], Name: representative_gene_method, dtype: int64)

In [20]:
ensg.gene_df.query("ensembl_gene_id != ensembl_representative_gene_id").head(2)

,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source,gene_biotype,gene_description,ensembl_source,ensembl_created_date,ensembl_modified_date,...,chromosome,seq_region_exc_type,seq_region,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,ensembl_representative_gene_id


# replaced ID converter

A single `old_stable_id` can map to multiple `new_stable_id`. For example, `ENSG00000152006`

https://uswest.ensembl.org/Homo_sapiens/Tools/IDMapper/Results?tl=AzhM62SpkvdiLC4H-6808613

Requested ID | Matched ID(s) | Releases
-- | -- | --
ENSG00000152006 | ENSG00000196273 | 26: ENSG00000196273.1
ENSG00000152006 | ENSG00000197016 | 26: ENSG00000197016.1
ENSG00000152006 | ENSG00000196239 | 26: ENSG00000196239.1

In [21]:
ensg.old_to_new_df.head(2)

,old_ensembl_gene_id,new_ensembl_gene_id
0,ENSRNOG00000000132,ENSRNOG00000031425
1,ENSRNOG00000000194,ENSRNOG00000031589


In [22]:
# some ensembl genes replaced by many new ensembl genes
ensg.old_to_new_df.old_ensembl_gene_id.value_counts().head(2)

ENSRNOG00000034138    202
ENSRNOG00000027512     82
Name: old_ensembl_gene_id, dtype: int64

In [23]:
# example
ensg._update_ensembl_gene("ENSG00000152006")

{'ENSG00000152006'}

In [24]:
ensg.old_to_newest_df.head(2)

,old_ensembl_gene_id,newest_ensembl_gene_id,is_current
0,ENSRNOG00000000132,ENSRNOG00000031425,True
1,ENSRNOG00000000194,ENSRNOG00000031589,False


In [25]:
len(ensg.old_to_newest_df)

19373

In [26]:
ensg.old_to_newest_df.is_current.value_counts()

True     14987
False     4386
Name: is_current, dtype: int64

## omni-updater

The omni-updater dataset is designed to convert ensembl gene IDs from input data to the current, representative ensembl_gene_ids for this ensembl release. It assumes:

- users want to update outdated genes with their replacements
- users want a dataset of representative genes only, and want to convert alternative alleles to representative genes

An inner join of a dataset with `update_df` on `input_ensembl_gene_id` will do the following:

- produce output ensembl_gene_ids that are current and representatives
- update outdated genes with their current identifiers. Outdated genes with no current replacement will be removed by the inner join.
- update alternative gene alleles with their representatives
- genes that are already represenative and current will map to themselves

In [27]:
ensg.update_df.head(2)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
0,ENSRNOG00000000001,ENSRNOG00000000001,True,True,1,1
1,ENSRNOG00000000007,ENSRNOG00000000007,True,True,1,1


In [28]:
ensg.update_df.sort_values("input_maps_to_n_genes", ascending=False).head(2)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
9523,ENSRNOG00000023620,ENSRNOG00000049190,False,True,93,37
8978,ENSRNOG00000025734,ENSRNOG00000046957,False,True,93,46


In [29]:
ensg.update_df.sort_values("n_inputs_map_to_gene", ascending=False).head(2)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
8295,ENSRNOG00000029700,ENSRNOG00000061111,False,True,16,130
7494,ENSRNOG00000025861,ENSRNOG00000054729,False,True,93,130


In [30]:
(ensg.update_df.input_maps_to_n_genes == 1).mean()

0.6935867975767704

In [31]:
ensg.update_df.query("ensembl_gene_id == 'ENSG00000256263'")

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene


In [32]:
print(
    f"The omni-updater contains {len(ensg.update_df):,} rows for mapping "
    f"{ensg.update_df.input_ensembl_gene_id.nunique():,} input genes to "
    f"{ensg.update_df.ensembl_gene_id.nunique():,} current, representative genes."
)

The omni-updater contains 47,870 rows for mapping 34,963 input genes to 32,883 current, representative genes.


In [33]:
# https://useast.ensembl.org/Homo_sapiens/Tools/IDMapper/Results?tl=P45VLMbogubpI0QA-6815464
ensg.update_df.query("input_ensembl_gene_id == 'ENSG00000201456'").head(3)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene


## cross-refrences (xrefs)

In [34]:
ensg.xref_df.head()

,ensembl_gene_id,xref_source,xref_accession,xref_label,xref_description,xref_info_type,xref_linkage_annotation
0,ENSRNOG00000000001,ArrayExpress,ENSRNOG00000000001,ENSRNOG00000000001,None,DIRECT,None
1,ENSRNOG00000000001,Clone_based_ensembl_gene,AABR07013255.1,AABR07013255.1,None,MISC,None
2,ENSRNOG00000000007,ArrayExpress,ENSRNOG00000000007,ENSRNOG00000000007,None,DIRECT,None
3,ENSRNOG00000000007,EntrezGene,24379,Gad1,glutamate decarboxylase 1,DEPENDENT,None
4,ENSRNOG00000000007,Reactome_gene,R-RNO-112310,R-RNO-112310,Neurotransmitter release cycle,DIRECT,None


In [35]:
# datasets where there are ensembl_gene_id-xref_source-xref_accession pairs might not be distinct 
xref_dup_df = ensg.xref_df[ensg.xref_df.duplicated(subset=["ensembl_gene_id", "xref_source", "xref_accession"], keep=False)]
xref_dup_df.xref_source.value_counts()

Series([], Name: xref_source, dtype: int64)

In [36]:
# xref sources versus info_types
pd.crosstab(ensg.xref_df.xref_source, ensg.xref_df.xref_info_type, margins=True)

xref_info_type,DEPENDENT,DIRECT,MISC,PROJECTION,All
xref_source,,,,,
ArrayExpress,0,32883,0,0,32883
Clone_based_ensembl_gene,0,0,9468,0,9468
Clone_based_vega_gene,0,0,857,0,857
EntrezGene,21996,0,0,0,21996
MGI,0,0,0,75,75
RFAM,0,2566,0,0,2566
RGD,297,22339,0,0,22636
Reactome_gene,0,82282,0,0,82282
Uniprot_gn,25248,0,0,0,25248


## Gene Ontology xrefs

In [37]:
ensg.xref_go_df.head(3)

,ensembl_gene_id,go_id,go_label,go_evidence_codes,xref_info_types,ensembl_transcript_ids,ensembl_representative_gene_id
0,ENSRNOG00000000007,GO:0003824,catalytic activity,IEA,"DEPENDENT,DIRECT","ENSRNOT00000000008,ENSRNOT00000087134,ENSRNOT0...",ENSRNOG00000000007
1,ENSRNOG00000000007,GO:0004351,glutamate decarboxylase activity,"IEA,ISO,ISS","PROJECTION,DIRECT","ENSRNOT00000000008,ENSRNOT00000084375,ENSRNOT0...",ENSRNOG00000000007
2,ENSRNOG00000000007,GO:0005737,cytoplasm,"IEA,ISO","PROJECTION,DIRECT","ENSRNOT00000000008,ENSRNOT00000087712",ENSRNOG00000000007


In [38]:
# GO terms for CCR5
# compare to http://useast.ensembl.org/Homo_sapiens/Gene/Ontologies/molecular_function?g=ENSG00000160791
sorted(ensg.xref_go_df.query("ensembl_gene_id == 'ENSG00000160791'").go_label)

[]

## lrg xrefs

In [39]:
ensg.xref_lrg_df.head(2)

,ensembl_gene_id,lrg_gene_id


In [40]:
len(ensg.xref_lrg_df)

0

### ncbigene xrefs

In [41]:
ensg.xref_ncbigene_df.head()

,ensembl_representative_gene_id,ncbigene_id
0,ENSRNOG00000000007,24379
1,ENSRNOG00000000008,296511
2,ENSRNOG00000000009,366158
3,ENSRNOG00000000010,498922
4,ENSRNOG00000000012,296272


In [42]:
# ensembl gene mapped to by multiple ncbigenes
ensg.xref_ncbigene_df.ensembl_representative_gene_id.value_counts().head(3)

ENSRNOG00000059366    13
ENSRNOG00000020119    11
ENSRNOG00000018740     8
Name: ensembl_representative_gene_id, dtype: int64

In [43]:
len(ensg.xref_ncbigene_df), ensg.xref_ncbigene_df.ensembl_representative_gene_id.duplicated().sum()

(21996, 1153)

In [44]:
# ncbigene mapped to by multiple ensembl genes, likely due to alt gene alleles
ensg.xref_ncbigene_df.ncbigene_id.value_counts().head(3)

364723       12
502913       12
100912564    12
Name: ncbigene_id, dtype: int64

In [45]:
len(ensg.xref_ncbigene_df), ensg.xref_ncbigene_df.ncbigene_id.duplicated().sum()

(21996, 663)

In [46]:
# ensg.xref_ncbigene_df.query("ensembl_representative_gene_id == 'ENSG00000231500'")
# ensg.xref_ncbigene_df.query("ncbigene_id == '51206'")

In [47]:
repr_ensembl_gene_ids = set(ensg.gene_df.ensembl_representative_gene_id)
len(repr_ensembl_gene_ids)

32883

In [48]:
# many of these genes should probably be alternative alleles rather than representative
ensg.gene_df.query("not primary_assembly and ensembl_gene_id==ensembl_representative_gene_id")

,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source,gene_biotype,gene_description,ensembl_source,ensembl_created_date,ensembl_modified_date,...,chromosome,seq_region_exc_type,seq_region,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,ensembl_representative_gene_id
